# Procesamiento de climatologías

Este _notebook_ contiene programas en Python para el procesamiento de climatologías. Una climatología se define como un conjunto de medidas estadísticas que resumen series temporales de variables climáticas como precipitación, temperatura, radiación solar y velocidad del viento, entre otras.

**Bibliotecas**

In [ ]:
import os

import rasterio

from shapely.geometry import Point

import pandas as pd
import geopandas as gpd

## Entrada

In [ ]:
# Directorio de datos de entrada
INPUT_DATA_DIR = "C:/Users/mfvargas/raster/raster_recortados/"

# Directorio de datos de salida
OUTPUT_DATA_DIR = "C:/Users/mfvargas/raster/raster_recortados/"

# Shapefile de salida
OUTPUT_SHAPEFILE = "Enero_1981-2010.shp"

# Valor de NO DATA
NO_DATA_VALUE = 65535

## Procesamiento

In [ ]:
# DataFrame
centros_gdf = gpd.GeoDataFrame(columns=['geometry', 'x', 'y'], crs={"init": "epsg:4326"})

# Contador de archivos
f = 0

# Recorrido de los archivos del directorio de entrada
for file in os.listdir(INPUT_DATA_DIR):
    anio = file[12:16]
    mes = file[17:19]
    
    # Condiciones de salida
    if mes != '01':
        continue
    
    #if anio != '1981':
    #    continue
    
    print("Año:", anio, "Mes:", mes)
    centros_gdf['c_' + anio + '_' + mes] = 65535
    
    dataset = rasterio.open(INPUT_DATA_DIR + "/" + file)
    print("Archivo:", file, "Ancho:", dataset.width, "Largo:", dataset.height)
    
    # Recuperación de los valores de la banda 1 en un arreglo
    band = dataset.read(1)
    
    # Recorrido de las filas y columnas de la banda
    i = 0
    t = 0
    for row in band:
        j = 0
        for value in row:
            if value != NO_DATA_VALUE:
                coordinates = dataset.transform * (j, i)
                x = coordinates[0]
                y = coordinates[1]
                point = Point(coordinates)
                print(t, i, j, x, y, point, value)
                if (f == 0):
                    centros_gdf.loc[t] = [point] + [x] + [y] + [value]
                else:
                    centros_gdf.loc[[t], ['c_' + anio + '_' + mes]] = value
                t += 1
            j += 1
            
        i += 1
    f += 1
    
# Columna con promedio
centros_gdf['promedio'] = centros_gdf.mean(axis=1)

## Salida

In [ ]:
centros_gdf

In [ ]:
centros_gdf.to_file(OUTPUT_DATA_DIR + "/" + OUTPUT_SHAPEFILE)